In [11]:
from PIL import Image

from api import Detector
from fisheye_utills import FisheyeUtills as FU

In [12]:
# weights can be downloaded from
# put weights to folder named "./weights/"
# https://github.com/duanzhiihao/RAPiD/releases/download/v0.1/pL1_MWHB608_Mar11_4500.ckpt
rapid = Detector(model_name='rapid',
                 weights_path='./weights/pL1_MWHB1024_Mar11_4000.ckpt',
                 use_cuda=False)

Using backbone Darknet-53. Loading ImageNet weights....
Number of parameters in backbone: 40584928
Successfully initialized model rapid. Total number of trainable parameters: 61523734
Successfully loaded weights: ./weights/pL1_MWHB1024_Mar11_4000.ckpt
Using CPU instead of CUDA...


In [13]:
def process(img_path, model):
    
    # should open image first
    img = Image.open(img_path)  
    # feed image to model
    detections = model.detect_one(pil_img=img,
                                  visualize=False, 
                                  input_size=1024, 
                                  conf_thres=0.4, 
                                  test_aug=None)
    # convert fisheye images
    if detections.shape[0] > 0:
        # make fisheye utills object
        img_utills = FU(img)
        # ignore confidence values at last element
        uvwha = detections[:,:5]
        # returns are described below
        # patch.shape     : [N, C, H, W]
        # sphericals.shape: [N, 2(lon, lat), H, W]
        # k_values.shape  : [N]
        patches, sphericals, k_values = img_utills.get_tangent_patch(uvwha,
                                                              visualize=False,
                                                              detectnet=True)
    else:
        patches = None
    return patches, k_values

patches = process(img_path='./images/lunch.jpg', model=rapid)